In [1]:
import boto3
import io
import re
import numpy as np
import os
import xml.etree.ElementTree as ET 
from joblib import Parallel, delayed
import multiprocessing
#from utilities.score_computors import are_sentences_alike
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

In [2]:
def extract_clean_str(original_str):
    return re.sub('[^a-zA-Z0-9]+', ' ', original_str, flags=re.UNICODE).lower().strip().replace(' ', '')

def get_mapped(path):
    key_map, dict_idx = {}, 0
    for r, _, f in os.walk(dir_path):
        for file in f:
            if '.JPG' in file or '.jpg' in file:
                key_map[dict_idx] = file
                dict_idx += 1
    idxs = list(key_map.keys())          
    shuffled_idxs = np.random.randint(0, len(idxs), len(idxs))
    return key_map, shuffled_idxs

def fetch_from_xml(path, file):
    tree = ET.parse(os.path.join(path, file)).getroot()  
    return extract_clean_str(''.join([e.attrib['char'] for e in tree.iter(tag='character')]))

In [ ]:
xml_file = 'annotations/captions_val2017.json'
# if not os.path.exists(os.path.join(dir_path, xml_file)):
#     return False
# ground_truth_str = fetch_from_xml(dir_path, xml_file)

In [7]:
import json
with open('annotations/instances_val2017.json') as json_file:
    caption = json.load(json_file)

In [15]:
caption['images'][20]

{'license': 1,
 'file_name': '000000143931.jpg',
 'coco_url': 'http://images.cocodataset.org/val2017/000000143931.jpg',
 'height': 480,
 'width': 640,
 'date_captured': '2013-11-16 15:38:46',
 'flickr_url': 'http://farm9.staticflickr.com/8474/8145963753_2e334946c7_z.jpg',
 'id': 143931}

In [43]:
captions, mapped_annotation = {}, {}

for annotation in caption['annotations']:
    mapped_annotation[annotation['image_id']] = annotation['caption']

for image in caption['images']:
    captions[image['file_name']] = mapped_annotation.get(image['id'])
        
mapped_annotation.clear()

In [ ]:
mapped_annotation = [ ]

In [31]:
len(mapped_annotation.keys())

5000

In [44]:
captions

{'000000397133.jpg': 'A table with pies being made and a person standing near a wall with pots and pans hanging on the wall.',
 '000000037777.jpg': 'A kitchen that has a bowl of fruit on the table.',
 '000000252219.jpg': 'People are walking on the street by a homeless person.',
 '000000087038.jpg': 'a man falls off his skateboard in a skate park.',
 '000000174482.jpg': 'a bike sits parked next to a street ',
 '000000403385.jpg': 'A very big whit rest room with a shabby looking shower.',
 '000000006818.jpg': 'This picture looks like a janitors closet with buckets on the floor.',
 '000000480985.jpg': 'The front end of a red motorcycle that is on display.',
 '000000458054.jpg': 'LOTS OF BROKEN TOILETS SITTING OUT ON A LAWN',
 '000000331352.jpg': 'Off white toilet with a faucet and controls. ',
 '000000296649.jpg': 'A view of motorcyclists riding their bikes through heavy city traffic.',
 '000000386912.jpg': 'a woman is sitting in front of a desk',
 '000000502136.jpg': 'A plaster external 

In [10]:
import json
with open('annotations/captions_val2017.json') as json_file:
    caption = json.load(json_file)

In [11]:
for items in caption['images']:
    if items['file_name'] == '000000523957.jpg':
        print (items)
        break

{'license': 4, 'file_name': '000000523957.jpg', 'coco_url': 'http://images.cocodataset.org/val2017/000000523957.jpg', 'height': 427, 'width': 640, 'date_captured': '2013-11-20 12:12:19', 'flickr_url': 'http://farm3.staticflickr.com/2079/2660283696_07138cec04_z.jpg', 'id': 523957}


In [12]:
for annotation in caption['annotations']:
    if annotation['image_id'] == 523957:
        print (annotation)
        break

{'image_id': 523957, 'id': 778146, 'caption': 'A guy is skateboarding through cones on a street.'}


In [23]:
caption['annotations'][0]['ima']

{'image_id': 179765,
 'id': 38,
 'caption': 'A black Honda motorcycle parked in front of a garage.'}

In [41]:
#caption

In [31]:
from nltk.corpus import wordnet

from itertools import product
sims = []

list1 = ['Compare', 'require']
list2 = ['choose', 'copy', 'define', 'duplicate', 'find', 'how', 'identify', 'label', 'list', 'listen', 'locate', 'match', 'memorise', 'name', 'observe', 'omit', 'quote', 'read', 'recall', 'recite', 'recognise', 'record', 'relate', 'remember', 'repeat', 'reproduce', 'retell', 'select', 'show', 'spell', 'state', 'tell', 'trace', 'write']
list = []

for word1, word2 in product(list1, list2):
    syns1 = wordnet.synsets(word1)
    syns2 = wordnet.synsets(word2)
    for sense1, sense2 in product(syns1, syns2):
        d = wordnet.wup_similarity(sense1, sense2)
        sims.append((d, syns1, syns2))

In [33]:
sims

[(None,
  [Synset('comparison.n.03'),
   Synset('compare.v.01'),
   Synset('compare.v.02'),
   Synset('compare.v.03'),
   Synset('compare.v.04')],
  [Synset('choose.v.01'), Synset('choose.v.02'), Synset('choose.v.03')]),
 (None,
  [Synset('comparison.n.03'),
   Synset('compare.v.01'),
   Synset('compare.v.02'),
   Synset('compare.v.03'),
   Synset('compare.v.04')],
  [Synset('choose.v.01'), Synset('choose.v.02'), Synset('choose.v.03')]),
 (None,
  [Synset('comparison.n.03'),
   Synset('compare.v.01'),
   Synset('compare.v.02'),
   Synset('compare.v.03'),
   Synset('compare.v.04')],
  [Synset('choose.v.01'), Synset('choose.v.02'), Synset('choose.v.03')]),
 (0.3333333333333333,
  [Synset('comparison.n.03'),
   Synset('compare.v.01'),
   Synset('compare.v.02'),
   Synset('compare.v.03'),
   Synset('compare.v.04')],
  [Synset('choose.v.01'), Synset('choose.v.02'), Synset('choose.v.03')]),
 (0.4,
  [Synset('comparison.n.03'),
   Synset('compare.v.01'),
   Synset('compare.v.02'),
   Synset('

In [75]:
from nltk.corpus import wordnet 
#syns = wordnet.synsets("program")
synonyms = []
for syn in wordnet.synsets("metal"): 
    for l in syn.lemmas(): 
        synonyms.append(l.name()) 
synonyms

['metallic_element', 'metal', 'alloy', 'metal', 'metal', 'metallic', 'metal']

In [39]:
from nltk.corpus import wordnet as wn
from nltk.corpus import wordnet_ic

dog=wn.synsets('dog', pos=wn.NOUN)[0] #get the first noun synonym of the word "dog"
cat=wn.synsets('cat', pos=wn.NOUN)[0]
rose=wn.synsets('rose', pos=wn.NOUN)[0]
flower=wn.synsets('flower', pos=wn.NOUN)[0]

In [43]:
import nltk
nltk.download('wordnet_ic')

[nltk_data] Downloading package wordnet_ic to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping corpora/wordnet_ic.zip.


True

In [44]:
brown_ic = wordnet_ic.ic('ic-brown.dat') 

In [66]:
bathroom=wn.synsets('food', pos=wn.NOUN)[0]
toilet=wn.synsets('ice', pos=wn.NOUN)[0]
print (bathroom.lin_similarity(toilet, brown_ic))

0.4220146244807438


NameError: name 'token' is not defined